# Running R Code from Julia

This notebook provides a basic introduction to how to run R code, using the [RCall.jl](https://github.com/JuliaInterop/RCall.jl) package, from Julia. See the the RCall.jl homepage (the installation instructions) for how to use your existing R installation or for letting RCall.jl install R for you.

In [1]:
using Printf, DelimitedFiles
include("src/printmat.jl");

# Load Data

In [2]:
x = readdlm("Data/MyData.csv",',',skipstart=1)  #reading the csv file

(Rme,Rf,R) = (x[:,2],x[:,3],x[:,4])  #creating variables from columns of x
y  = R - Rf                          #do R .- Rf if R has several columns

c = ones(length(Rme))
x = [c Rme]

b = x\y
println("OLS coeffs according to Julia")
printmat(b)

OLS coeffs according to Julia
    -0.504
     1.341



# R

The next few cells load the RCall.jl package, transfers the data from Julia to R (`@rput`) and then run an OLS regression (`reval()`).

In [3]:
using RCall

In [4]:
@rput x y                          #send x and y to R

resultsR = reval("summary(mod <- lm(y ~ x-1))")  #run R code
println(resultsR)                                #and print output

RObject{VecSxp}

Call:
lm(formula = y ~ x - 1)

Residuals:
    Min      1Q  Median      3Q     Max 
-17.981  -3.131  -0.359   2.281  52.361 

Coefficients:
   Estimate Std. Error t value Pr(>|t|)    
x1 -0.50416    0.30483  -1.654    0.099 .  
x2  1.34105    0.06573  20.401   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 5.954 on 386 degrees of freedom
Multiple R-squared:  0.5194,	Adjusted R-squared:  0.5169 
F-statistic: 208.6 on 2 and 386 DF,  p-value: < 2.2e-16




In [5]:
println(names(resultsR))           #print all keys (field names)

[:call, :terms, :residuals, :coefficients, :aliased, :sigma, :df, Symbol("r.squared"), Symbol("adj.r.squared"), :fstatistic, Symbol("cov.unscaled")]


In [6]:
b_R = rcopy(resultsR[:coefficients])  #the numerical results are now a Julia array

printblue("Comparing the estimates in Julia and R")
printmat([b b_R[:,1]];colNames=["Julia","R"])

Comparing the estimates in Julia and R
     Julia         R
    -0.504    -0.504
     1.341     1.341



In [7]:
R"model <- lm(y ~ x-1)"                              #an alternative approach
resultsR2 = rcall(:summary, R"model")
b_R2      = rcopy(resultsR2[:coefficients])
printmat([b b_R[:,1] b_R2[:,1]];colNames=["Julia","R","R ver, 2"])

     Julia         R  R ver, 2
    -0.504    -0.504    -0.504
     1.341     1.341     1.341



In [8]:
#we can also run blocks of code like this
#do @rput x y or use $x and $y in the code below

#solve x'x*b = x'y
R"""
xx <- t(x)%*%x
xy <- t(x)%*%y
b_R <- solve(xx,xy)
"""

RObject{RealSxp}
           [,1]
[1,] -0.5041626
[2,]  1.3410486


### Interactive Usage of R

If you are using the Julia REPL ("command prompt"), then you can work more interactively with R. Unfortunaterly, that does not work in a notebook. The following trivial example illustrates the idea (`>julia` means that you are at the julia prompt and `R>` at the R prompt).

```
julia> x = 1;y=2
julia> @rput x y
julia> $         #to switch over to R
R> z <- x + y
R> hit ESC       #to switch over to Julia
julia> @rget z
```

At this point you can work with `z` in Julia.